In [2]:
import os
import cv2
import joblib
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
def extract_color_features(image):
    hist_b = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist_g = cv2.calcHist([image], [1], None, [256], [0, 256])
    hist_r = cv2.calcHist([image], [2], None, [256], [0, 256])
    color_features = np.concatenate((hist_b, hist_g, hist_r))
    return color_features

In [4]:
def extract_texture_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp_image = local_binary_pattern(gray, P=8, R=1, method='uniform')
    lbp_hist, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, 59), range=(0, 58))
    return lbp_hist

In [5]:
def extract_features(data):
    color_features = []
    texture_features = []
    extracted_features = []

    for image in data:
        image = cv2.resize(image, (128, 128))
        color_feats = extract_color_features(image)
        color_features.append(color_feats)

        texture_feats = extract_texture_features(image)
        texture_features.append(texture_feats)

    for i in range(len(data)):
        color_flat = color_features[i].ravel()
        texture_flat = texture_features[i].ravel()

        combined = np.concatenate((color_flat, texture_flat))
        extracted_features.append(combined)

    return extracted_features

In [6]:
dataset = 'Fruit Dataset'
data = []
labels = []
classes = ['good', 'bad', 'mixed']
fruit_names = ['apple', 'guava', 'banana', 'lemon', 'orange', 'pomegranate']
batch_size = 32

In [7]:
def image_generator(dataset, classes, fruit_names, batch_size):
    data = []
    labels = []

    for class_name in classes:
        for fruit_name in fruit_names:
            image_folder = os.path.join(dataset, class_name, fruit_name)
            for image in os.listdir(image_folder):
                image_path = os.path.join(image_folder, image)
                if os.path.isfile(image_path):
                    image = cv2.imread(image_path)
                    data.append(image)
                    labels.append(class_name)

                    if len(data) == batch_size:
                        yield data, labels
                        data = []
                        labels = []

    if data:
        yield data, labels

In [8]:
X_train = []
y_train = []

for data_batch, labels_batch in image_generator(dataset, classes, fruit_names, batch_size):
    extracted_features = extract_features(data_batch)
    X_train.extend(extracted_features)
    y_train.extend(labels_batch)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
joblib.dump(scaler, 'scaler.h5')

['scaler.h5']

In [11]:
model = SVC(kernel='linear')
model.fit(X_train, y_train)
joblib.dump(model, 'model.h5')

In [ ]:
predictions = model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy*100:.2f}%')

Accuracy: 86.33%
